In [6]:
import praw # Python Reddit API Wrapper
#Praw documentation: https://praw.readthedocs.io/en/stable/index.html
import pandas as pd


# Create a Reddit instance
# This is a read-only instance since we only need to read data
# We don't need it to be authenticated since we aren't posting anything
reddit = praw.Reddit(
    client_id='51f3IBDpHGRzaGh0FGj0ng',
    client_secret='C-mvx1K5Flj_4kM-Grc9FLT2_bHxag',
    user_agent='TradingAnalysisApp by /u/Ok_Ability_9624',
)



Version 7.8.0 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


In [7]:
print(reddit.read_only)

True


In [8]:
# Fetch and print the titles of the top 10 hot posts in the 'test' subreddit
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)

Some test commands
Test post
spoilers
Join the Blackhawk Defense Force – We Need You! [MILSIM]
Test
test img 2
Test
Tryna see how this emoji shows
test
Ok so since somebody told me to come here I’m coming here


In [9]:
subreddit = reddit.subreddit("wallstreetbets")
print(subreddit.display_name)  
print(subreddit.title)
print(subreddit.description)

wallstreetbets
wallstreetbets
The rules and submission guidelines are maintained on new Reddit so be sure to check them and make sure you're up to date.

* Read the [rules](https://www.reddit.com/r/wallstreetbets/about/rules)

* Read the [comment and submission guide](https://www.reddit.com/r/wallstreetbets/wiki/contentguide)

* Read the [FAQ](https://www.reddit.com/r/wallstreetbets/wiki/faq) if you're new to both wallstreetbets and trading.

---
**Join the discord**

[WSB Discord](https://discord.gg/wsbverse)

**filter by flairs**

^Navigate ^WSB |^We ^recommend ^best ^daily ^DD
:--|:--     
**DD** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restrict_sr=on&q=flair%3ADD) / [**Best Daily**](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=day) / [Best Weekly](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=week)
**Discussion** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restric

In [66]:

stockCSV = pd.read_csv('stock_info.csv') #Read the CSV fil

stockCSV['Ticker'] = '$' + stockCSV['Ticker'] #Add a $ sign in front of the ticker
stockCSV.head() #Display the first 5 rows of the dataframe


,Ticker,Name,Exchange
0,$A,Agilent Technologies,NYSE
1,$AA,Alcoa Inc.,NYSE
2,$AAN,Aaron's Inc,NYSE
3,$AAT,American Assets Trust,NYSE
4,$AAV,Advantage Oil & Gas Ltd,NYSE


In [ ]:

postData = []
import nltk  #Natural Language Toolkit
nltk.download('punkt') #sentence word tokenziation
import textstat #Ta simple python library to calculate statistics from text 
from textblob import TextBlob
import datetime
buySell = 0
# Fetch the top 1000 hot posts in the 'wallstreetbets' subreddit
for submission in subreddit.hot(limit=1000):
        textWords = nltk.word_tokenize(submission.selftext) #tokenize the post text
        words = submission.title.split() #split the title into words
        blob = TextBlob(submission.selftext) #create a textblob object
        
        #determining buy sell depending on 
        if(blob.sentiment.polarity >0):
            buySell = 1
        elif(blob.sentiment.polarity <0):
            buySell = -1
        else:
            buySell = 0
        
        if(submission.selftext == ''): #skip any posts that contain no text
            continue
        elif any(word in stockCSV['Ticker'].values for word in words): #check if any of the words in the title are stock abbreviations
            postData.append({
                 #These are indentifying data
                'stock abbreviation': next(word for word in words if word in stockCSV['Ticker'].values),
                'stock name': stockCSV.loc[stockCSV['Ticker'] == next(word for word in words if word in stockCSV['Ticker'].values)]['Name'].values[0],
                'title': submission.title,
                'date': datetime.datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'id': submission.id,
                'url': submission.url, 
                'post text': submission.selftext, #post text
                #All of the following are features that can be used to predict the stock price
                'upvote Ratio': submission.upvote_ratio, #upvote ratio
                'comment number': submission.num_comments, #number of comments
                'text length': len(submission.selftext),#length of the post text
                'flesh_kincaid_grade': textstat.flesch_kincaid_grade(submission.selftext), #Flesch Kincaid Grade
                'lexical_diversity': len(set(textWords)) / len(textWords), #lexical diversity
                'Sentiment (polarity):': blob.sentiment.polarity, #Sentiment
                'Sentiment (subjectivity)': blob.sentiment.subjectivity, #Subjectivity
                'Buy/Sell': buySell #Buy or Sell
            })
        else:
            continue
        

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [90]:
df = pd.DataFrame(postData)
df


,stock abbreviation,stock name,title,date,id,url,post text,upvote Ratio,comment number,text length,flesh_kincaid_grade,lexical_diversity,Sentiment (polarity):,Sentiment (subjectivity),Buy/Sell
0,$TSLA,Tesla Motors,Thank you Elon! $TSLA $10k to $149k in under a...,2024-11-11 18:28:35,1gp6l38,https://i.redd.it/k519nqm4xc0e1.jpeg,Opened a lotto account last week after the ele...,0.84,196,129,4.0,0.884615,0.175000,0.308333,1
1,$TSLA,Tesla Motors,"$80,000 to $1.12M in 7 days. Thank you Elon Mu...",2024-11-11 10:34:23,1gouy6r,https://www.reddit.com/r/wallstreetbets/commen...,https://preview.redd.it/5i2pyzrxja0e1.jpg?widt...,0.70,1240,821,10.4,0.692810,0.160859,0.390825,1
2,$TSLA,Tesla Motors,"Time to inverse WSB, 50k puts on $TSLA",2024-11-11 14:37:05,1gp0z1i,https://i.redd.it/f0dbekftrb0e1.png,You are all so bullish on TSLA. Time to do the...,0.96,341,144,3.4,0.909091,0.000000,0.000000,0
3,$TSLA,Tesla Motors,$TSLA Calls Closed +$400k,2024-11-11 18:38:58,1gp6tf7,https://www.reddit.com/r/wallstreetbets/commen...,Haven't gotten any sleep holding these Tesla c...,0.93,46,561,10.3,0.750000,0.005000,0.620000,1
4,$COST,Costco Wholesale Corp.,Caught some of the $COST move Friday 🤑,2024-11-09 17:22:42,1gnlme2,https://i.redd.it/up8nesxjbyzd1.jpeg,"Beautiful breakout to ATH, could done somethin...",0.96,62,102,6.8,1.000000,0.850000,1.000000,1
5,$TSLA,Tesla Motors,"$TSLA 0DTEs $2850 -> $40,300",2024-11-08 10:56:51,1gmlso4,https://i.redd.it/j0921dts9pzd1.jpeg,Bought last night before close. Expected a $30...,0.96,279,289,2.2,0.900000,-0.078333,0.395000,0
6,$TSLA,Tesla Motors,$TSLA $70 -> $2800 +4000%,2024-11-08 19:53:56,1gmy2zw,https://www.reddit.com/gallery/1gmy2zw,Placed a limit order on lotto calls and went t...,0.92,77,170,13.4,0.888889,0.160269,0.429630,1
7,$QQQ,Powershares QQQ,$QQQ options AKA fucked,2024-11-07 17:01:38,1gm2g5a,https://www.reddit.com/gallery/1gm2g5a,"Okay guys, I screwed up. I trade options from ...",0.50,15,702,6.3,0.629870,0.108333,0.308333,1
8,$TSLA,Tesla Motors,Another $TSLA election bet,2024-11-06 15:07:27,1gl7qqr,https://i.redd.it/bvw8hvro8czd1.jpeg,I figured $TSLA calls would either go to the m...,0.87,16,194,3.5,0.928571,0.361905,0.345238,1
9,$SMCI,Super Micro Computer,Chat am I cooked? $SMCI 💀,2024-11-05 21:54:33,1gkolej,https://i.redd.it/x13efxme47zd1.jpeg,Sitting in my room looking at this while the w...,0.72,62,107,7.2,0.954545,0.250000,0.400000,1
